Ceci n'est pas un commentaire


# ***Google Captcha image recognition***
**A Deep Learning Project using TensorFlow**

*by Emma Begard, Augustin Bouveau, Bagin Jobert-Rollin, Hugues Boisdon*

## I Data **Fetching**

Our source Dataset can be found at : *https://www.kaggle.com/datasets/mikhailma/test-dataset*

*credits : **Mike Mazurov***

### I.1 **Dowloading** Data Files from ***KaggleHub*** Source

In [102]:
import kagglehub

DATA_FOLDER_PATH_IF_CACHED = kagglehub.dataset_download("mikhailma/test-dataset")
print("Path to dataset files in cache:", DATA_FOLDER_PATH_IF_CACHED)

Path to dataset files in cache: C:\Users\Development\.cache\kagglehub\datasets\mikhailma\test-dataset\versions\1


The data files are downloaded in the Users **cache** by default.
If the data source folder is moved, ***please update*** the following *path variable*

In [103]:
DATA_FOLDER_PATH_IF_MOVED = "" # Data folder path if data was moved since download

### I.2 **Loading** Data

#### I.2.A *Functions*

In [104]:
import os
import pandas as pd
from PIL import Image


def loadImagesFromDirectory(directoryPath:str) -> pd.DataFrame:
    data: dict[str:list] = {"Image": [], "Label": []}
    
    for root, directories, files in os.walk(directoryPath):
        for f in files:
            subFolderName = f.split(" ")[0]
            if subFolderName == "Cross"  : subFolderName = "Crosswalk"
            if subFolderName == "Tlight" : subFolderName = "Traffic Light"

            data["Image"].append(Image.open(directoryPath+ "/" + subFolderName + "/"+ f).convert("RGB"))
            data["Label"].append(subFolderName)

    return pd.DataFrame(data)
 


#### I.2.B Loading **raw** image data in a *Dataframe*

In [105]:
imgFolderPath = DATA_FOLDER_PATH_IF_CACHED+"/Google_Recaptcha_V2_Images_Dataset/images" if DATA_FOLDER_PATH_IF_MOVED == "" else DATA_FOLDER_PATH_IF_MOVED+"/Google_Recaptcha_V2_Images_Dataset/images"

rawImagesDf = loadImagesFromDirectory(imgFolderPath)
rawImagesDf.head(5)

,Image,Label
0,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
1,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
2,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
3,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle
4,<PIL.Image.Image image mode=RGB size=120x120 a...,Bicycle


## II Data **Preprocessing**

### II.1 **Cleaning** Data

#### II.1.A *Images Size* Normalization 

##### II.1.A.a funcs

In [106]:
def checkImagesSizeInDataframe(df:pd.DataFrame) -> None:
    sizeCounts = {}
    for image in df["Image"]:
        if not(image.size in sizeCounts):
            sizeCounts[image.size] = 1
        else :
            sizeCounts[image.size] += 1
    print(f"Sizes found in Dataframe : {sizeCounts}")

In [113]:
from PIL import ImageOps
from copy import deepcopy

def normalizeImageSizes(df:pd.DataFrame, imgTargetSize:tuple[int,int] = (120, 120)) -> pd.DataFrame:
    dataframe = deepcopy(df)
    numOfCropped = 0; numOfExpandeds = 0
    for index, row in dataframe.iterrows():
        width  = row["Image"].width
        height = row["Image"].height
        
        noPaddings  = [0   , 0  ,     0,      0]
        noCroppings = [0   , 0  , width, height]
                    #  Left, Top, Right, Bottom
        
        paddings  = deepcopy(noPaddings)
        croppings = deepcopy(noCroppings)
        
        if width < imgTargetSize[0]:
            pixelsToAdd = imgTargetSize[0] - width
            paddings[0] = pixelsToAdd  // 2
            paddings[2] = (pixelsToAdd - pixelsToAdd  // 2)
        elif width > imgTargetSize[0]:
            pixelsToCrop = width - imgTargetSize[0]
            croppings[0] = pixelsToCrop // 2
            croppings[2] = width - (pixelsToCrop-pixelsToCrop // 2)
        
        if height < imgTargetSize[1]:
            pixelsToAdd = imgTargetSize[1] - height
            paddings[1] = pixelsToAdd  // 2
            paddings[3] = (pixelsToAdd - pixelsToAdd  // 2)
        elif height > imgTargetSize[1]:
            pixelsToCrop = height - imgTargetSize[1]
            croppings[1] = pixelsToCrop // 2
            croppings[3] = height - (pixelsToCrop-pixelsToCrop // 2)
        
        if paddings != noPaddings:
            row["Image"] = ImageOps.expand(row["Image"], border=tuple(paddings), fill='black')
            numOfExpandeds += 1
        if croppings != noCroppings:
            row["Image"] = ImageOps.crop(row["Image"], border=tuple(paddings))
            numOfCropped += 1
    print(f"{numOfCropped} images were cropped to {imgTargetSize}!")
    print(f"{numOfExpandeds} images were expanded to {imgTargetSize}!")
    return dataframe


##### II.1.A.b Normalizing process

In [126]:
# Initial Check
checkImagesSizeInDataframe(rawImagesDf)

normalizedSizeImagesDf = normalizeImageSizes(rawImagesDf)

# Final Check
checkImagesSizeInDataframe(normalizedSizeImagesDf)

Sizes found in Dataframe : {(120, 120): 10705, (100, 100): 1025}
0 images were cropped to (120, 120)!
1025 images were expanded to (120, 120)!
Sizes found in Dataframe : {(120, 120): 11730}


#### II.1.B Transforming Images to *pixel value Arrays*

In [123]:
from copy import deepcopy
import numpy as np

def imagesToPixelArrays(df:pd.DataFrame) -> pd.DataFrame:
    dataframe = deepcopy(df)
    for id, row in dataframe.iterrows():
        row["Image"] = np.array(row["Image"]).astype(np.float32)
    return dataframe

In [127]:
pixelArraysDf = imagesToPixelArrays(normalizedSizeImagesDf)

pixelArraysDf.head(5)

,Image,Label
0,"[[[117.0, 114.0, 104.0], [114.0, 110.0, 101.0]...",Bicycle
1,"[[[54.0, 75.0, 67.0], [58.0, 76.0, 69.0], [66....",Bicycle
2,"[[[38.0, 33.0, 27.0], [37.0, 34.0, 28.0], [37....",Bicycle
3,"[[[49.0, 49.0, 51.0], [55.0, 56.0, 58.0], [72....",Bicycle
4,"[[[135.0, 128.0, 125.0], [143.0, 130.0, 129.0]...",Bicycle


#### II.1.C *Pixel values* Normalization

In [125]:
def normalizePixelValues(df:pd.DataFrame) -> pd.DataFrame:
    dataframe = deepcopy(df)
    for id, row in dataframe.iterrows():
        row["Image"] /= 255
    return dataframe

In [128]:
normalizedPixelArraysDf = normalizePixelValues(pixelArraysDf)

normalizedPixelArraysDf.head(5)

,Image,Label
0,"[[[0.45882353, 0.44705883, 0.40784314], [0.447...",Bicycle
1,"[[[0.21176471, 0.29411766, 0.2627451], [0.2274...",Bicycle
2,"[[[0.14901961, 0.12941177, 0.105882354], [0.14...",Bicycle
3,"[[[0.19215687, 0.19215687, 0.2], [0.21568628, ...",Bicycle
4,"[[[0.5294118, 0.5019608, 0.49019608], [0.56078...",Bicycle


### II.2 **Splitting** Data in *Training* and *Testing* 

In [ ]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(df["Image"].to_numpy(), df["Label"].to_numpy(), test_size=0.2, random_state=42)

train_images, test_images = train_images / 255.0, test_images / 255.0

print(train_images[:5])
print(train_labels[:5])
print(test_images[:5])
print(test_labels[:5])

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
print(train_images)

In [ ]:
print(type(train_images))

<class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf

#for index, row in train_images.iterrow():
#    print(row)

#train_images = train_images.values.reshape(-1,1)
#print(train_images[:5])
train_images = tf.convert_to_tensor(train_images, dtype=np.float32)
print(train_images)

## III **Designing** the *Model*

In [ ]:
from keras import layers


def build_model(dense_neurons):
    model = tf.keras.Sequential([
        layers.InputLayer((120,120,3)),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(dense_neurons, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
model = build_model(dense_neurons=64)
BAT = 64
    # Train the model
history = model.fit(
    train_images,train_labels,
    batch_size=BAT,
    epochs=5,
    verbose=2
)
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels, verbose=2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

How to load images with tensorflow  : https://www.tensorflow.org/api_docs/python/tf/keras/utils/load_img

In [ ]:
tf.keras.utils.load_img(
    folderPath,
    color_mode='rgb',
    target_size=None,
    interpolation='nearest',
    keep_aspect_ratio=False
)